In [2]:
from sympy.combinatorics import Permutation
import numpy as np
import scipy.sparse as scpy

In [3]:
def perform_action(state, action):
    """
    perform the given action on the given state in-place
    for twisty puzzles this means applying the permutations of a move

    inputs:
    -------
        state - (list) of (int) - list representing the state
        action - (list) of (list) of (int) - list representing an action, here as a list of cycles
            cycles are lists of list indices of the state list.
    """
    for cycle in action: # loop over all cycles in the move
        j = cycle[0]
        for i in cycle:  # apply cycle
            state[i], state[j] = state[j], state[i]
    return state

In [4]:
def get_perm_matrix(perm, n):
    """
    convert a permutation in cyclic notation into a permutation matrix, such that M@s = perm(s)
    inputs:
        perm - (list) of (tuple) of (int) - permutation in cyclic notation
        n - (int) - number of elements in permutation
    returns:
        (np.array) - permutation matrix
    """
    M = np.eye(n, dtype=np.bool_)
    # M = np.eye(n, dtype=np.uint8)
    for cycle in perm: # loop over all cycles in the move
        j = cycle[0]
        # M[j, i] 
        for i in cycle:  # apply cycle
            c = np.array(list(M[:,j]))
            M[:,j] = M[:,i]
            M[:,i] = c
            # M[:,i], M[:,j] = M[:,j], M[:,i]
    return M

In [5]:
def test_my_perm(state, perm):
    """
    perform a permutation `perm`, given as a (list) of (tuple)s of (int)s, on a python (list) `state` using pure python loops and list operations.
    `perm` represents the cyclic notation of a permutation, each tuple is one cycle.
    """
    state = perform_action(state, perm)

def test_sp_perm(state, perm):
    """
    perform a sympy (Permutation) `perm` on a python (list) `state`
    """
    state = perm(state)

def test_np_perm(state, perm_matrix):
    """
    perform a permutation given by a permutation matrix (np.array) of (int) on `state`, also given as a (np.array) of (int)
    """
    state = perm_matrix@state

In [20]:
def time_py_sp_np(perm, n=25):
    state = list(range(n))
    state_sp = list(range(n))
    state_np = np.array(state, dtype=np.uint8)
    perm_sp = Permutation(perm, size=n)
    perm_np = get_perm_matrix(perm, n)
    perm_scpy = scpy.csr_matrix(perm_np)
    %timeit test_my_perm(state, perm)
    %timeit test_sp_perm(state_sp, perm_sp)
    %timeit test_np_perm(state_np, perm_np)
    %timeit test_np_perm(state_np, perm_scpy)

In [23]:
Permutation??

Init signature: Permutation(*args, **kwargs)
Source:        
class Permutation(Atom):
    """
    A permutation, alternatively known as an 'arrangement number' or 'ordering'
    is an arrangement of the elements of an ordered list into a one-to-one
    mapping with itself. The permutation of a given arrangement is given by
    indicating the positions of the elements after re-arrangement [2]_. For
    example, if one started with elements [x, y, a, b] (in that order) and
    they were reordered as [x, y, b, a] then the permutation would be
    [0, 1, 3, 2]. Notice that (in SymPy) the first element is always referred
    to as 0 and the permutation uses the indices of the elements in the
    original ordering, not the elements (a, b, etc...) themselves.

    >>> from sympy.combinatorics import Permutation
    >>> from sympy.interactive import init_printing
    >>> init_printing(perm_cyclic=False, pretty_print=False)

    Permutations Notation

    Permutations are commonly represented i

In [21]:
perm1 = [[1,2,3], [4,5,0]]
perm2 = [[1,2], [3,4,5,0], [6,7,8,9,10,11], [12,13], [14,15,16], [17,18,19,20,21,22,23,24]]
perm3 = [[0,1,2], [4,6,7], [8,9,11,12,13,14], [16,17,19,20,21,22,23], [24,25,26,27,28,30], [31,32,34,40,41,42], [43,44,46], [48,49,50,52,54,55], [56,57,58,59,60,62,64,65,67,69,70,72,73,74,75], [78,79,80,82,83,84,85,86], [87,89,90], [91,92,93,94,95,96,97,98,99], [299,300]]

In [22]:
# time_py_sp_np(perm1, n=6)
time_py_sp_np(perm2, n=100)
# time_py_sp_np(perm3, n=800)

2.23 µs ± 48.1 ns per loop (mean ± std. dev. of 7 runs, 100000 loops each)
20.1 µs ± 587 ns per loop (mean ± std. dev. of 7 runs, 100000 loops each)
18.4 µs ± 1.62 µs per loop (mean ± std. dev. of 7 runs, 10000 loops each)
10.1 µs ± 2.64 µs per loop (mean ± std. dev. of 7 runs, 100000 loops each)


In [71]:
perm = perm1
n = 6

state = list(range(n))
state_sp = list(range(n))
state_np = np.array(state, dtype=np.uint8)
perm_sp = Permutation(perm)
perm_np = get_perm_matrix(perm, n)
print(perform_action(state, perm))
print(perm_sp(state_sp))
print(perm_np@state_np)

[5, 3, 1, 2, 0, 4]
[4, 2, 3, 1, 5, 0]
[4. 2. 3. 1. 5. 0.]
